# Inference of the SEIR model

For background of the purpose of this model, please refer to this [exercise sheet](https://benlambertdotcom.files.wordpress.com/2020/10/week_project.docx).

## Task 1: Code up the ODE and inference model and use these to simulate an epidemic.

In [1]:
import numpy as np

import seirmo


# Define times and parameters
times = np.arange(start=0, stop=40, step=1)  # time in day
initial_values = [0.9, 0, 0.1, 0]
beta = 0.5  # 50% of infected and susceptible entcounters per day lead to transmission
kappa = 0.5  # incubation rate
gamma = 0.05  # recovery rate

# Collect all parameters
parameters = initial_values + [beta] + [kappa] + [gamma]

# Simulate model
model = seirmo.SEIRModel()
result = model.simulate(parameters, times, return_incidence=True)

In [2]:
import plotly.graph_objects as go

# Plot I: Create plot of S, E, I, R
fig = go.Figure()

# Plot of susceptibles
fig.add_trace(
    go.Scatter(
        x=times,
        y=result[:, 0],
        mode='lines',
        name='Susceptible'
    )
)

# Plot of exposed
fig.add_trace(
    go.Scatter(
        x=times,
        y=result[:, 1],
        mode='lines',
        name='Exposed'
    )
)

# Plot of infectious
fig.add_trace(
    go.Scatter(
        x=times,
        y=result[:, 2],
        mode='lines',
        name='Infectious'
    )
)

# Plot of Recovered
fig.add_trace(
    go.Scatter(
        x=times,
        y=result[:, 3],
        mode='lines',
        name='Recovered'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time in day',
    yaxis_title='Percentage in population'
)

fig.show()

# Plot II: Create plot of incidence number per day
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=result[:, 4],
        name='Incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time in day',
    yaxis_title='Incidence number in population percentage'
)

fig.show()

## Task 2: Using either an optimiser or sampling algorithm, estimate the parameters of the model.

### Generate fake data

In [34]:
# Generate fake data
standard_deviation = 0.005
noise = np.random.normal(loc=0, scale=standard_deviation, size=len(times))
incidence_data = result[:, 4] + noise

# Set negative incidences to zero
mask = incidence_data < 0
incidence_data[mask] = np.zeros(shape=len(incidence_data[mask]))

# Visualise data
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=incidence_data,
        name='Incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time in day',
    yaxis_title='Incidence number in population percentage'
)

fig.show()

### Infer model parameters

In [35]:
import pints

class SEIRModel(pints.ForwardModel):
    def __init__(self):
        super(SEIRModel, self).__init__()

        self._model = seirmo.SEIRModel()

    def n_parameters(self):
        # Returns number of parameters, i.e. 4 initial conditions and 3 parameters
        return 7

    def n_output(self):
        # Returns number of model outputs.
        # We will only return the incidence number, because this is what we
        # have "measurements" for.
        return 1

    def simulate(self, log_parameters, times):
        output = self._model.simulate(
            parameters=np.exp(log_parameters), times=times, return_incidence=True)
        n_incidence = output[:, 4]

        return n_incidence

In [36]:
# Create starting points of samplers
true_parameters = np.array([0.9, 0, 0.1, 0, 0.5, 0.5, 0.1])

# Shift starting point a little bit just to make it a little bit harder
initial_params = np.log(true_parameters + 0.1)

# Create log-likelihood
pints_model = SEIRModel()
problem = pints.SingleOutputProblem(pints_model, times, incidence_data)
log_likelihood = pints.GaussianKnownSigmaLogLikelihood(problem, sigma=0.005)

# Run sampling routines
optimiser = pints.OptimisationController(
    function=log_likelihood,
    x0=initial_params,
    method=pints.CMAES
)

log_estimates, _ = optimiser.run()
estimates = np.exp(log_estimates)

Maximising LogPDF
Using Covariance Matrix Adaptation Evolution Strategy (CMA-ES)
Running in sequential mode.
Population size: 9
Iter. Eval. Best      Time m:s
0     9     -66.07995   0:00.0
1     18    -23.3096    0:00.1
2     27     55.15446   0:00.1
3     36     75.56947   0:00.1
20    189    155.8729   0:00.7
40    369    158.4988   0:01.2
60    549    158.5628   0:01.8
80    729    158.6517   0:02.3
100   909    158.6679   0:02.9
120   1089   158.6692   0:03.5
140   1269   158.6692   0:04.1
160   1449   158.6692   0:04.7
180   1629   158.6692   0:05.3
200   1809   158.6692   0:05.8
220   1989   158.6692   0:06.4
240   2169   158.6692   0:07.0
260   2349   158.6692   0:07.5
280   2529   158.6692   0:08.1
300   2709   158.6693   0:08.6
320   2889   158.6693   0:09.2
340   3069   158.6693   0:09.7
360   3249   158.6693   0:10.3
380   3429   158.6693   0:10.9
400   3609   158.6693   0:11.4
420   3789   158.6693   0:12.0
440   3969   158.6693   0:12.6
460   4149   158.6693   0:13.1
480 

In [37]:
print('True parameters:') 
print(true_parameters)
print(' ')
print('Estimates:')
print(estimates)

True parameters:
[0.9 0.  0.1 0.  0.5 0.5 0.1]
 
Estimates:
[8.85870479e-01 2.03186450e-02 8.72774964e-02 2.21664011e+02
 7.32082456e-01 2.47027745e-01 2.04196986e-02]


### Compare inferred model to data

In [40]:
true_incidences = result[:, 4]
inferred_incidences = pints_model.simulate(log_parameters=log_estimates, times=times)

# Plot data and inferred incidences
fig = go.Figure()

# Plot of data
fig.add_trace(
    go.Bar(
        x=times,
        y=true_incidences,
        name='True incidences'
    )
)

# Plot of data
fig.add_trace(
    go.Bar(
        x=times,
        y=inferred_incidences,
        name='Inferred incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time in day',
    yaxis_title='Incidence number in population percentage'
)

fig.show()